# 1. DATA CLEANING #

Lending Club releases data for all of the approved and declined loan applications periodically [on their website](https://www.lendingclub.com/info/download-data.action). 
A data dictionary, describing the columns of the dataset, can be found [here](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097).

Since declined applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on **LoanStats** that describes approved loans datasets.

We'll work with approved loans data from 2007 to 2011.

In [1]:
#reading in to Pandas
import pandas as pd
pd.options.display.max_columns = 0

loans = pd.read_csv("loans.csv", low_memory=False)
print(loans.shape)
loans.head()

(42538, 52)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Now, let's try to clean up the data by removing irrelevant columns. These columns contain leaking or redundant information that are not helpful to an investor.

I'm going to split the columns set into 3 groups and use the [documentation](https://app.dataquest.io/m/133/machine-learning-project-walkthrough%3A-data-cleaning/4/first-group-of-columns) to detect irrelevant columns:
- First group: from `id` to `purpose`.
- Second group: from `title` to `total_rec_prncp`.
- Third group: the rest of the data.

## First group of columns  ##

Thanks to the documentation, I can detect leaking and redundant information from the following columns:
- `id`: randomly generated field by Lending CLub (LC) for unique identification purposes only.
- `member_id`: also a randomly generated field by LC for unique identification purposes only.
- `funded_amnt` and `funded_amnt_inv`: leak data from the future (after the loan is already started to be funded).
- `grade` and `sub_grade`: contains redundant information as the interest rate column (`int_rate`).
- `emp_title`: requires other data and a lot of processing to potentially be useful.
- `issue_d`: leaks data from the future (after the loan is already completely funded).

Therefore, I will remove all of this columns.

In [2]:
loans = loans.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d'], axis=1)
len(loans.columns)

44

## Second group of columns ##

Within this group, I am going to drop these columns as follows:
- `zip_code`: redundant with the `addr_state` column since only the first 3 digits of the 5 digit zip code are visible.
- `out_prncp` and `out_prncp_inv`: leak data from the future.
- `total_pymnt`, `total_pymnt_inv`: also leak data from the future.
- `total_rec_prncp`: also leak data from the future.

In [3]:
loans = loans.drop(['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp'], axis=1)
len(loans.columns)

38

## Third group of columns##

In the last group of columns, I will drop these irrelevant columns that leak data from the future:
- `total_rec_int`
- `total_rec_late_fee`
- `recoveries` 
- `collection_recovery_fee`
- `last_pymnt_d`
- `last_pymnt_amnt`

In [4]:
loans = loans.drop(['total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 
                    'last_pymnt_d', 'last_pymnt_amnt'], axis=1)
len(loans.columns)

32

## Target column##

In order to process a model, we need a target column. The most likely column is `loan_status`, since it's the only one that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. 

In [5]:
loans['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

From the investor's perspective, I am interested in trying to predict which loans will be paid off on time and which ones won't. Only the **Fully Paid** and **Charged Off** values meet the criteria. The other values describes loans that are still on going and thus cannot say anything about the outcome of the loans. While the "Default" status resembles the "Charged Off" status, in LC's eyes, loans that are charged off have essentially NO CHANCE of being repaid, whereas defautl loans have a SMALL CHANCE.

Consequently, I will only keep "Fully Paid" and "Charged Off" values and transform the column using **binary classification**, where "Fully Paid" = 1 and "Charged Off" = 0.  

In [6]:
loans = loans[(loans['loan_status'] == 'Fully Paid') | (loans['loan_status'] == 'Charged Off')]

mapping = {
    'loan_status':{
        'Fully Paid': 1,
        'Charged Off': 0
    }
}

loans = loans.replace(mapping)

loans['loan_status'].value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

## Finally, removing single value columns##

The single value columns won't be useful for the model since they don't add any information to each loan application. They need to be removed as well.

In [7]:
single_value_col = list()

for col in loans.columns:
    non_null = loans[col].dropna()
    unique_count = len(non_null.unique())
    if unique_count == 1:
        single_value_col.append(col)
        
loans = loans.drop(single_value_col, axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Sep-2013,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,Jun-2016,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,Jan-2016,0.0


# 2. PREPARING THE FEATURES#

Before selecting the features, I'm dealing with missing values and numerical data transformation first, these are the 2 conditions for model processing.

## Handling missing values##

In [8]:
loans.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

While most of the columns have no missing values, three columns have 50 or less missing values, and two columns, `emp_length` and `pub_rec_bankruptcies` have a relatively high amount of missing values.emp_length

Domain knowledge tells us that employment length is used in assessing how risky a potential borrower is, if a borrower hadn't had a steady employment, the risk of not being repaid would have been high. So I'll keep the column `emp_length` despite its relatively large amount of missing values.

Let's inspect the values of the column `pub_rec_bankruptcies`:

In [9]:
loans['pub_rec_bankruptcies'].value_counts(normalize=True, dropna=False)

 0.0    0.939438
 1.0    0.042456
NaN     0.017978
 2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64

In terms of variability, nearly 94% of values are in the same category, it probably won't have much predictive value, so I will drop it.

Additionally, I'll drop the missing values in the others columns as well.

In [10]:
loans = loans.drop('pub_rec_bankruptcies', axis=1).dropna()

## Converting text columns to categorical ones##

In [11]:
loans.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In [12]:
# select only object columns from the data set
object_df = loans.select_dtypes(include=['object'])
object_df.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


According to the documentation, I got the following information:
- `term`: number of payments on the loan, either 36 or 60 ==> 2 unique values
- `home_ownership`: home ownership status, provided by the borrower during registration, 4 unique values: RENT, OWN, MORTGAGE, OTHER.

To make sure there are no other unique values I will check as well.

For the following columns, I need to check the number of unique values:
- `verification_status`: indicates if income was verified by LC.
- `emp_length`: number of years the borrower was employed upon time of application.
- `addr_state`: borrower's state of residence.

The `purpose` and `title`, based on their first rows' values, could reflect the same information, that means it could bring them to a high linearity, so one of them could have been removed. Let's check the unique value counts separately to confirm.

The `int_rate` and `revol_util` represent numeric values and need to be converted.

Finally, `earliest_cr_line` and `last_credit_pull_d` contain date values that would require a good amount of feature engineering for them to be potentially useful. Consequently, I will drop these two.

In [13]:
# explore unique values in the 5 categorical columns
cols = ['term', 'home_ownership', 'verification_status', 'emp_length', 'addr_state']

for col in cols:
    print(loans[col].value_counts())
    print("-----------------------------------------------------------")

 36 months    28234
 60 months     9441
Name: term, dtype: int64
-----------------------------------------------------------
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
-----------------------------------------------------------
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
-----------------------------------------------------------
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
-----------------------------------------------------------
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV    

Since the `emp_length` values are ordinals (1 year < 2 years < 3 years ...), I will clean it up and treat it as a numerical one.

In [14]:
mapping = {
    'emp_length': {
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0
    }
}

loans = loans.replace(mapping)
loans['emp_length'].value_counts()

10    8545
0     4513
2     4303
3     4022
4     3353
5     3202
1     3176
6     2177
7     1714
8     1442
9     1228
Name: emp_length, dtype: int64

The `addr_state` contains so much unique values that can significantly slow down the model. I decide to drop it.

In [15]:
loans = loans.drop('addr_state', axis=1)

In [16]:
# comparing unique values in both 'title' and 'purpose' columns
print(loans['title'].value_counts())
print('---------------------------------------')
print(loans['purpose'].value_counts())

Debt Consolidation                         2068
Debt Consolidation Loan                    1599
Personal Loan                               624
Consolidation                               488
debt consolidation                          466
Credit Card Consolidation                   345
Home Improvement                            336
Debt consolidation                          314
Small Business Loan                         298
Credit Card Loan                            294
Personal                                    290
Consolidation Loan                          250
Home Improvement Loan                       228
personal loan                               219
Loan                                        202
Wedding Loan                                199
personal                                    198
Car Loan                                    188
consolidation                               186
Other Loan                                  168
Wedding                                 

It seems like the `purpose` and `title` columns do contain overlapping information but we'll keep the `purpose` column since it contains much fewer discrete values and, in fact, the unique values in `purpose` do cover many unique values in `title`. 

In [17]:
loans = loans.drop('title', axis=1)

In [18]:
# transform 'int_rate' and 'revol_util' to numerical type
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype(float)
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype(float)

In [19]:
# finally, drop the date columns
loans = loans.drop(['last_credit_pull_d', 'earliest_cr_line'], axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


The last job is to convert categorical colums into dummy ones. After getting dummy columns, I will drop the original non-dummy columns.

In [20]:
to_dummy = ['term', 'home_ownership', 'verification_status', 'purpose']

loans = pd.concat([
    loans,
    pd.get_dummies(loans[to_dummy])
], axis=1
).drop(to_dummy, axis=1)

loans.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null int64
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
term_ 36 months            

# 3. MAKING PREDICTIONS#

Looking back to the target column, `loan_status`.

In [22]:
loans['loan_status'].value_counts()

1    32286
0     5389
Name: loan_status, dtype: int64

We notice that there is an imbalance, with about 6 times as many loans that were paid off on time (value = `1`) than those that weren't (value = `0`). Imbalance can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. So I need to **keep the imbalance issue in mind** when building machine learning models.

## Picking an error metric##

Back to the problematic question I wanted to answer:

**"Can we build a machine learning (ML) model that can accurately predict if a borrower will pay off their loan on time or not ?"**

My objectif in this is to make money: I want to fund enough loans that are paid off on time to offset my losses from loans that aren't paid off. A good error metric will help me determine if my algorithm will make me money or lose me money.

So in this case, let's see what'd happen between my predictions and the actual loan status:


|`loan_status`|`prediction`|Result              |Translation                     | 
|-------------|-------------|--------------------|--------------------------------|
|1|1|true positive|I predict that a loan will be paid off on time, and it actually is. So I'd gain money|
|0|0|true negative|I predict that a loan won't be paid off on time, and it actually isn't. So I'd save my money|
|1|0|false negative|I predict that a loan won't be paid off on time, but it actually is. So I'd lose an opportunity to make money|
|0|1|false positive|I predict that a loan will be paid off on time, but it actually isn't. So I'd lose money|

With a conservative investor's perspective, **I'd want to minimize risk, and avoid false positives as much as possible**. It would be more okay with missing out on opportunites (false negatives) than it would be with funding a risky loan (false positives).

I'm going to do some calculations on these 4 situations.

In [23]:
# suppose that my prediction is that all of these loans will be paid off on time, so I set all predicting values = 1
import numpy as np
predictions = pd.Series(np.ones(len(loans)))

# find the number of true positives
tp_filter = (predictions == 1) & (loans['loan_status'] == 1)
tp = len(predictions[tp_filter])
print("true positives :", tp)

# find the number of true negatives
tn_filter = (predictions == 0) & (loans['loan_status'] == 0)
tn = len(predictions[tn_filter])
print("true negatives :", tn)

# find the number of false positives
fp_filter = (predictions == 1) & (loans['loan_status'] == 0)
fp = len(predictions[fp_filter])
print("false positives :", fp)

# find the number of false negatives
fn_filter = (predictions == 0) & (loans['loan_status'] == 1)
fn = len(predictions[fn_filter])
print("false negatives :", fn)

true positives : 30512
true negatives : 0
false positives : 5065
false negatives : 0


**The imbalance problem**

In [24]:
# let's see how accurate the predictions I set as above
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(loans['loan_status'], predictions)
accuracy

0.8569608493696085

In this case, my predictions are 85.7% accurate, that's means I've correctly identified `loan_status` in 85.7% of case.

**However**, recall that there is an **imbalance of 6 : 1** between `1` and `0`(or 6 : 1 between "Fully Paid" and "Charge Off"). This causes a major issue when I use accuracy as a metric. This is because of the imbalance, the classifier can predict `1` for every row, and with the "quite high accuracy, 85.7%", I could end up pulling all my money out for lending.

The rate false positives : true positives = 1 : 6, that means **for every 6 times I gain money, there is always 1 time that I lose money**. And imagine that I'm going to lend \$1000 on evarage with 10\% interest on each borrower. Let's see what happens.

|`loan_status`|`prediction`|Lending|Repayment|Interest back|Profit/Loss|
|-------------|------------|-------|---------|-------------|-----------|
|1             |1            |-1000       |   1000    |   100      |100           |
|1             |1            |-1000       |   1000  |     100        |100           |
|1             |1            |-1000       |   1000  |     100            |100           |
|1             |1            |-1000       |   1000      | 100            |100           |
|1             |1            |-1000       |   1000      | 100            |100           |
|1             |1            |-1000       |   1000      | 100            |100           |
|0             |1            |-1000       |   0      | 0            |-1000           |
|           -   |-            |-       |    -     | **Profit/Loss**            |**-400**           |

As you can see, I made 600 dollars in interest from the payoff money, but I lost 1000 dollars on the one borrower who aren't able to pay me back, even though my model is technically accurate.

Consequently, I will adjust my error metric accordingly, by optimizing for high true positive rate (or high **sensitivity, probability of detection**) and for low false positive rate (or low **fall-out, probability of false alarm**).

In [25]:
# calculate the sensitivity
tpr = tp / (tp + fn) 
print('sensitivity : ', tpr)

# calculate the fall-out
fpr = fp / (fp + tn) 
print('fall_out : ', fpr)

sensitivity :  1.0
fall_out :  1.0


tpr = fpr = 100% means that I correctly identified all of the good loans (true positive rate), but I also incorrectly identified all of the bad loans (false positive rate).

Now that I've set up error metrics, let's move on to making predictions using machine learning algorithms. 

## Logistic Regression##

Let's try out with logistic regression model, the upsides of using this one are that:
- it's quick to train and I can iterate more quickly;
- it's less prone to overfitting than more complex models;
- it's easy to interpret.

To avoid overfitting, I will use k-fold cross validation as well.

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# create a model instance with "balanced class weight" to keep the imbalance 6:1 compensated
lr = LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=1000, random_state=1)

# splitting features dataframe and target column
features = loans[loans.columns.drop('loan_status')]
target = loans['loan_status']

# make predictions with 3 fold cross validation
lr.fit(features, target)
predictions = cross_val_predict(lr, features, target, cv=3)

# turn predictions into pandas Series
predictions = pd.Series(predictions)

# recalculate the sensitivity and the fall-out
tn_filter = (predictions == 0) & (target == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) & (target == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) & (target == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) & (target == 0)
fp = len(predictions[fp_filter])

tpr = tp / (tp + fn)
print('sensitivity : ', tpr)

fpr = fp / (fp + tn)
print('fall_out : ', fpr)

sensitivity :  0.5120280545359203
fall_out :  0.5032576505429418


This time, let's manually set the class weight by imposing more "weight" to the `0` side.

In [27]:
penalty = {
    0: 10,
    1: 1
}
# create a model instance with the penalty
lr = LogisticRegression(class_weight=penalty, solver='lbfgs', max_iter=1000)

# make predictions with 3 fold cross validation
lr.fit(features, target)
predictions = cross_val_predict(lr, features, target, cv=3)

# turn predictions into pandas Series
predictions = pd.Series(predictions)

# recalculate the sensitivity and the fall-out
tn_filter = (predictions == 0) & (target == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) & (target == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) & (target == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) & (target == 0)
fp = len(predictions[fp_filter])

tpr = tp / (tp + fn)
print('sensitivity : ', tpr)

fpr = fp / (fp + tn)
print('fall_out : ', fpr)

sensitivity :  0.15528316727844782
fall_out :  0.1504442250740375


## Random forest##

Now let's try with random forest model to see if the result is improved.

In [28]:
from sklearn.ensemble import RandomForestClassifier

# create a model instance
rf = RandomForestClassifier(class_weight='balanced')

# make predictions with 3 fold cross validation
rf.fit(features, target)
predictions = cross_val_predict(rf, features, target, cv=3)

# turn predictions into pandas Series
predictions = pd.Series(predictions)

# recalculate the sensitivity and the fall-out
tn_filter = (predictions == 0) & (target == 0)
tn = len(predictions[tn_filter])

tp_filter = (predictions == 1) & (target == 1)
tp = len(predictions[tp_filter])

fn_filter = (predictions == 0) & (target == 1)
fn = len(predictions[fn_filter])

fp_filter = (predictions == 1) & (target == 0)
fp = len(predictions[fp_filter])

tpr = tp / (tp + fn)
print('sensitivity : ', tpr)

fpr = fp / (fp + tn)
print('fall_out : ', fpr)

C:\Users\Vinh Phuc\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


sensitivity :  0.9652595700052439
fall_out :  0.9676209279368213


It seems like the random forest is not effective in this case, due to the heavy weight of value `1`.

For instance, our best model is Logistic Regression with manual setting of class weight, giving 15.5% of sensitivity and 15% of fall-out. 